In [ ]:
## This file implements neural networks with/without dropout and regularizer for p003pkpresabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [89]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [90]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p003pkpresabsSTCC_qual.csv')
df.shape

(253, 870)

In [91]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [92]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
      ..
248    0
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [93]:
df.head()

,id,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,...,AACTAGGGGGGATTAGAATGCAAAATAAGTCAAAATCGCCTTTTAAAATTGCATTTTCTAAATTTATTCATAATAAAATTGCAATGTTATCGGTTATTTT,AACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTTCATTAATTT,AAATTTTTCAACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTT,AAATTTTACTTAACAACTAGTACATAAAGTAATACAATTAAATTAATTCTATCTGAAAGATGTGTGGGGCATCGTTATTTTAGGTGGATATGAGCAATTT,AAATTGCGTATTT,AAATCTTAGACGTAAACATGATAAAATGGCCTTGATTACTCAATAGTTATATTTCGGAGAACTGATTTGTGATATGATATTAAAGACTATAGGAGGATTT,AAATCTACTGTTT,ST,CC,pheno
0,107,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,1,5,5,0
1,109,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,1,8,8,0
2,115,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,5,5,1
3,120335,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,1,5,5,0
4,120337,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,1,5,5,0


In [94]:
df['pheno'].value_counts()

0    224
1     26
2      3
Name: pheno, dtype: int64

In [95]:
df_clean = df.drop(columns=['id'])

In [96]:
df_clean.shape

(253, 869)

In [97]:
df_clean.head()

,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,TTTTTGAAAATAATCATTAGCTTGCTCACTATATAATTTGATGAATATATTTCGTGAAAGTGGGTATTTATTTAATGATTATTCTATATATGATAGTATA,...,AACTAGGGGGGATTAGAATGCAAAATAAGTCAAAATCGCCTTTTAAAATTGCATTTTCTAAATTTATTCATAATAAAATTGCAATGTTATCGGTTATTTT,AACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTTCATTAATTT,AAATTTTTCAACAAACTCAATTAATACGAATTATCGCTTTCAATAAAAATTATTCATTAAATCATTAAAGATATTGAGTTCCAATACTATTTTCACTTTT,AAATTTTACTTAACAACTAGTACATAAAGTAATACAATTAAATTAATTCTATCTGAAAGATGTGTGGGGCATCGTTATTTTAGGTGGATATGAGCAATTT,AAATTGCGTATTT,AAATCTTAGACGTAAACATGATAAAATGGCCTTGATTACTCAATAGTTATATTTCGGAGAACTGATTTGTGATATGATATTAAAGACTATAGGAGGATTT,AAATCTACTGTTT,ST,CC,pheno
0,1,1,1,0,0,0,1,1,1,0,...,0,0,0,0,0,0,1,5,5,0
1,1,1,1,0,0,0,1,1,1,0,...,0,0,0,0,0,0,1,8,8,0
2,1,1,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,5,5,1
3,1,1,1,0,0,0,1,1,1,0,...,0,0,0,0,0,0,1,5,5,0
4,1,1,1,0,0,0,1,1,1,0,...,0,0,0,0,0,0,1,5,5,0


In [98]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 869) (253,)


In [99]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 224), (1, 224), (2, 224)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [100]:
############# Fully-Connected Neural Network ################

In [101]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [102]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [103]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [104]:
dat

,0,test
0,NRS265,1
1,GA984,0
2,NRS119,0
3,NRS249,1
4,NRS255,2
...,...,...
197,NRS035,1
198,NRS387,1
199,NRS222,0
200,BCH-SA-11,1


In [105]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 325us/step - loss: 3.5887 - accuracy: 0.4979 - val_loss: 2.0695 - val_accuracy: 0.6139
Epoch 2/100
470/470 [==============================] - 0s 134us/step - loss: 1.1737 - accuracy: 0.5745 - val_loss: 0.9642 - val_accuracy: 0.7475
Epoch 3/100
470/470 [==============================] - 0s 215us/step - loss: 0.6577 - accuracy: 0.7447 - val_loss: 0.5074 - val_accuracy: 0.7624
Epoch 4/100
470/470 [==============================] - 0s 178us/step - loss: 0.4556 - accuracy: 0.8085 - val_loss: 0.4051 - val_accuracy: 0.8614
Epoch 5/100
470/470 [==============================] - 0s 192us/step - loss: 0.3994 - accuracy: 0.8511 - val_loss: 0.6045 - val_accuracy: 0.8465
Epoch 6/100
470/470 [==============================] - 0s 175us/step - loss: 0.4259 - accuracy: 0.8574 - val_loss: 0.4950 - val_accuracy: 0.8564
Epoch 7/100
470/470 [==============================] - 0s 180us/step - loss: 0.3773 

Epoch 57/100
470/470 [==============================] - 0s 131us/step - loss: 0.1131 - accuracy: 0.9553 - val_loss: 0.2308 - val_accuracy: 0.9208
Epoch 58/100
470/470 [==============================] - 0s 129us/step - loss: 0.1173 - accuracy: 0.9574 - val_loss: 0.1932 - val_accuracy: 0.9406
Epoch 59/100
470/470 [==============================] - 0s 126us/step - loss: 0.1164 - accuracy: 0.9617 - val_loss: 0.2057 - val_accuracy: 0.9356
Epoch 60/100
470/470 [==============================] - 0s 104us/step - loss: 0.1115 - accuracy: 0.9489 - val_loss: 0.1925 - val_accuracy: 0.9406
Epoch 61/100
470/470 [==============================] - 0s 100us/step - loss: 0.1422 - accuracy: 0.9468 - val_loss: 0.2117 - val_accuracy: 0.9257
Epoch 62/100
470/470 [==============================] - 0s 100us/step - loss: 0.1445 - accuracy: 0.9404 - val_loss: 0.2257 - val_accuracy: 0.9257
Epoch 63/100
470/470 [==============================] - 0s 100us/step - loss: 0.1244 - accuracy: 0.9574 - val_loss: 0.1794 -

In [21]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

202/202 [==============================] - 0s 89us/step
over-sampling test accuracy: 94.55%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([1, 0, 0, 1, 2, 1, 1, 1, 2, 0, 1, 0, 1, 0, 1, 2, 0, 0, 2, 2, 2, 1,
       0, 0, 1, 1, 1, 2, 2, 1, 2, 2, 0, 2, 1, 0, 0, 2, 1, 0, 2, 1, 2, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 1, 0, 0,
       1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 1, 0, 2, 2, 0, 2, 1, 2, 2, 1, 1, 0,
       2, 1, 2, 2, 0, 0, 2, 2, 1, 1, 0, 0, 1, 2, 0, 0, 2, 1, 2, 2, 1, 1,
       1, 1, 2, 0, 2, 0, 0, 0, 2, 0, 0, 1, 1, 2, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 2, 2, 1, 1, 2, 0, 0, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 1, 0, 2,
       1, 0, 2, 1, 0, 0, 2, 0, 2, 1, 2, 0, 2, 1, 2, 1, 0, 2, 1, 1, 2, 0,
       1, 2, 2, 1, 0, 2, 2, 2, 1, 1, 1, 2, 0, 0, 1, 1, 2, 0, 1, 1, 1, 1,
       1, 0, 1, 2])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,NRS265,1,1
1,GA984,0,0
2,NRS119,0,0
3,NRS249,1,1
4,NRS255,2,2
...,...,...,...
197,NRS035,1,1
198,NRS387,1,1
199,NRS222,0,0
200,BCH-SA-11,1,1


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,0.001061,0.997622,1.316916e-03
1,0.996951,0.003033,1.668960e-05
2,0.999995,0.000005,3.724804e-12
3,0.005938,0.994062,5.165093e-08
4,0.000325,0.002290,9.973850e-01
...,...,...,...
197,0.002477,0.996072,1.451060e-03
198,0.036935,0.963009,5.596347e-05
199,0.993051,0.006947,2.236100e-06
200,0.006751,0.993249,6.406792e-09


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p003ST.csv", index = False,
         header=None)

In [25]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 203us/step - loss: 0.0900 - accuracy: 0.9638 - val_loss: 0.2009 - val_accuracy: 0.9307
Epoch 2/100
470/470 [==============================] - 0s 125us/step - loss: 0.0847 - accuracy: 0.9723 - val_loss: 0.2067 - val_accuracy: 0.9208
Epoch 3/100
470/470 [==============================] - 0s 107us/step - loss: 0.0953 - accuracy: 0.9617 - val_loss: 0.2308 - val_accuracy: 0.9010
Epoch 4/100
470/470 [==============================] - 0s 111us/step - loss: 0.0944 - accuracy: 0.9574 - val_loss: 0.1780 - val_accuracy: 0.9554
Epoch 5/100
470/470 [==============================] - 0s 122us/step - loss: 0.0849 - accuracy: 0.9596 - val_loss: 0.2641 - val_accuracy: 0.9109
Epoch 6/100
470/470 [==============================] - 0s 162us/step - loss: 0.0908 - accuracy: 0.9660 - val_loss: 0.2035 - val_accuracy: 0.9455
Epoch 7/100
470/470 [==============================] - 0s 137us/step - loss: 0.0830 

Epoch 57/100
470/470 [==============================] - 0s 124us/step - loss: 0.0744 - accuracy: 0.9638 - val_loss: 0.2181 - val_accuracy: 0.9307
Epoch 58/100
470/470 [==============================] - 0s 204us/step - loss: 0.0681 - accuracy: 0.9723 - val_loss: 0.2095 - val_accuracy: 0.9307
Epoch 59/100
470/470 [==============================] - 0s 187us/step - loss: 0.0730 - accuracy: 0.9702 - val_loss: 0.2035 - val_accuracy: 0.9356
Epoch 60/100
470/470 [==============================] - 0s 229us/step - loss: 0.0699 - accuracy: 0.9723 - val_loss: 0.2274 - val_accuracy: 0.9257
Epoch 61/100
470/470 [==============================] - 0s 228us/step - loss: 0.0703 - accuracy: 0.9723 - val_loss: 0.2079 - val_accuracy: 0.9406
Epoch 62/100
470/470 [==============================] - 0s 246us/step - loss: 0.0854 - accuracy: 0.9596 - val_loss: 0.2122 - val_accuracy: 0.9406
Epoch 63/100
470/470 [==============================] - 0s 498us/step - loss: 0.0797 - accuracy: 0.9723 - val_loss: 0.2122 -

In [26]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 96.94%


In [106]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [107]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [108]:
y_prob = df_proba[df_proba['phage']=='p003pkpresabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[1.06127840e-03, 9.97621830e-01, 1.31691610e-03],
       [9.96950750e-01, 3.03264220e-03, 1.66895950e-05],
       [9.99995100e-01, 4.92194570e-06, 3.72480430e-12],
       [5.93764850e-03, 9.94062360e-01, 5.16509250e-08],
       [3.25131720e-04, 2.28993800e-03, 9.97384970e-01],
       [3.94247500e-06, 9.99981760e-01, 1.42963910e-05],
       [3.59399270e-04, 7.57661340e-01, 2.41979350e-01],
       [4.46977400e-02, 9.55213960e-01, 8.82108300e-05],
       [3.25131720e-04, 2.28993800e-03, 9.97384970e-01],
       [9.99105400e-01, 8.90887800e-04, 3.71495570e-06],
       [1.05186690e-02, 9.89481330e-01, 1.17181720e-08],
       [9.98851660e-01, 1.14834250e-03, 3.89766500e-11],
       [3.69354300e-02, 9.63008700e-01, 5.59634720e-05],
       [9.99195750e-01, 6.42081100e-04, 1.62104130e-04],
       [5.65948100e-03, 9.92046360e-01, 2.29416830e-03],
       [3.25131720e-04, 2.28993800e-03, 9.97384970e-01],
       [9.77454660e-01, 2.25453400e-02, 7.24812650e-10],
       [6.83751800e-01, 3.16216

In [109]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [110]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.9858485351022664

In [111]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [112]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.9858485351022664

In [113]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [114]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [115]:
dat2

,0,test
0,CFBRSa05,0
1,NRS114,0
2,NRS168,1
3,NRS255,2
4,NRS209,2
...,...,...
197,NRS196,0
198,NRS255,2
199,NRS249,1
200,NRS209,2


In [116]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [31]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 353us/step - loss: 2.8825 - accuracy: 0.4851 - val_loss: 1.0783 - val_accuracy: 0.4406
Epoch 2/100
470/470 [==============================] - 0s 136us/step - loss: 0.8292 - accuracy: 0.6511 - val_loss: 0.5338 - val_accuracy: 0.7723
Epoch 3/100
470/470 [==============================] - 0s 228us/step - loss: 0.4555 - accuracy: 0.8085 - val_loss: 0.4792 - val_accuracy: 0.7673
Epoch 4/100
470/470 [==============================] - 0s 152us/step - loss: 0.4124 - accuracy: 0.8447 - val_loss: 0.4709 - val_accuracy: 0.8267
Epoch 5/100
470/470 [==============================] - 0s 148us/step - loss: 0.4148 - accuracy: 0.8660 - val_loss: 0.4795 - val_accuracy: 0.8366
Epoch 6/100
470/470 [==============================] - 0s 185us/step - loss: 0.4293 - accuracy: 0.8489 - val_loss: 0.3709 - val_accuracy: 0.8663
Epoch 7/100
470/470 [==============================] - 0s 193us/step - loss: 0.3428 

Epoch 57/100
470/470 [==============================] - 0s 191us/step - loss: 0.1692 - accuracy: 0.9383 - val_loss: 0.1872 - val_accuracy: 0.9356
Epoch 58/100
470/470 [==============================] - 0s 167us/step - loss: 0.1427 - accuracy: 0.9447 - val_loss: 0.1994 - val_accuracy: 0.9208
Epoch 59/100
470/470 [==============================] - 0s 125us/step - loss: 0.1209 - accuracy: 0.9574 - val_loss: 0.2017 - val_accuracy: 0.9158
Epoch 60/100
470/470 [==============================] - 0s 140us/step - loss: 0.1122 - accuracy: 0.9617 - val_loss: 0.1550 - val_accuracy: 0.9653
Epoch 61/100
470/470 [==============================] - 0s 139us/step - loss: 0.1156 - accuracy: 0.9638 - val_loss: 0.2011 - val_accuracy: 0.9010
Epoch 62/100
470/470 [==============================] - 0s 178us/step - loss: 0.1191 - accuracy: 0.9596 - val_loss: 0.1887 - val_accuracy: 0.9109
Epoch 63/100
470/470 [==============================] - 0s 145us/step - loss: 0.1317 - accuracy: 0.9511 - val_loss: 0.1687 -

In [34]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

202/202 [==============================] - 0s 170us/step
over-sampling test accuracy: 91.09%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([0, 0, 1, 2, 2, 2, 2, 1, 2, 2, 0, 1, 0, 0, 2, 1, 0, 2, 1, 2, 2, 2,
       0, 2, 0, 0, 2, 2, 2, 2, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 2, 0, 1, 0,
       2, 0, 1, 2, 2, 2, 1, 0, 0, 2, 2, 1, 1, 2, 0, 2, 0, 1, 1, 0, 1, 1,
       1, 2, 1, 2, 0, 1, 2, 1, 0, 0, 1, 1, 2, 1, 1, 2, 0, 0, 1, 0, 0, 1,
       1, 2, 0, 1, 2, 1, 1, 1, 0, 2, 0, 1, 1, 1, 0, 2, 2, 1, 0, 0, 1, 0,
       2, 0, 1, 1, 1, 2, 0, 2, 1, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 2, 1, 1,
       1, 0, 1, 1, 1, 1, 2, 2, 0, 2, 0, 2, 2, 0, 1, 2, 0, 0, 1, 2, 2, 2,
       0, 2, 1, 1, 0, 2, 2, 2, 1, 1, 0, 2, 1, 2, 0, 0, 2, 0, 2, 2, 2, 1,
       1, 1, 2, 1, 2, 0, 1, 0, 1, 2, 0, 2, 1, 0, 0, 0, 1, 1, 2, 2, 1, 0,
       2, 1, 2, 0])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,CFBRSa05,0,0
1,NRS114,0,0
2,NRS168,1,1
3,NRS255,2,2
4,NRS209,2,2
...,...,...,...
197,NRS196,0,0
198,NRS255,2,2
199,NRS249,1,1
200,NRS209,2,2


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,7.180933e-01,0.281888,1.898983e-05
1,9.799658e-01,0.020034,3.792795e-10
2,7.303862e-02,0.926957,4.478878e-06
3,3.421833e-04,0.002073,9.975851e-01
4,3.214109e-12,0.000843,9.991568e-01
...,...,...,...
197,9.960671e-01,0.003932,9.774284e-07
198,3.421833e-04,0.002073,9.975851e-01
199,2.347543e-02,0.976525,3.907105e-09
200,3.214109e-12,0.000843,9.991568e-01


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p003ST.csv", index = False,
         header=None)

In [38]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 187us/step - loss: 0.1114 - accuracy: 0.9617 - val_loss: 0.1984 - val_accuracy: 0.9356
Epoch 2/100
470/470 [==============================] - 0s 176us/step - loss: 0.1130 - accuracy: 0.9553 - val_loss: 0.2100 - val_accuracy: 0.9356
Epoch 3/100
470/470 [==============================] - 0s 155us/step - loss: 0.1097 - accuracy: 0.9596 - val_loss: 0.2527 - val_accuracy: 0.9059
Epoch 4/100
470/470 [==============================] - 0s 150us/step - loss: 0.1126 - accuracy: 0.9574 - val_loss: 0.2132 - val_accuracy: 0.9356
Epoch 5/100
470/470 [==============================] - 0s 125us/step - loss: 0.1128 - accuracy: 0.9574 - val_loss: 0.2323 - val_accuracy: 0.9307
Epoch 6/100
470/470 [==============================] - 0s 118us/step - loss: 0.1122 - accuracy: 0.9617 - val_loss: 0.2146 - val_accuracy: 0.9158
Epoch 7/100
470/470 [==============================] - 0s 120us/step - loss: 0.1130 

Epoch 57/100
470/470 [==============================] - 0s 228us/step - loss: 0.0964 - accuracy: 0.9638 - val_loss: 0.2162 - val_accuracy: 0.9406
Epoch 58/100
470/470 [==============================] - 0s 190us/step - loss: 0.0910 - accuracy: 0.9660 - val_loss: 0.2480 - val_accuracy: 0.9307
Epoch 59/100
470/470 [==============================] - 0s 181us/step - loss: 0.0936 - accuracy: 0.9660 - val_loss: 0.2401 - val_accuracy: 0.9307
Epoch 60/100
470/470 [==============================] - 0s 159us/step - loss: 0.0957 - accuracy: 0.9617 - val_loss: 0.2214 - val_accuracy: 0.9356
Epoch 61/100
470/470 [==============================] - 0s 235us/step - loss: 0.0962 - accuracy: 0.9638 - val_loss: 0.2114 - val_accuracy: 0.9406
Epoch 62/100
470/470 [==============================] - 0s 272us/step - loss: 0.1084 - accuracy: 0.9447 - val_loss: 0.2068 - val_accuracy: 0.9406
Epoch 63/100
470/470 [==============================] - 0s 154us/step - loss: 0.1083 - accuracy: 0.9617 - val_loss: 0.2959 -

In [39]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 96.16%


In [117]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [118]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [119]:
y_prob2 = df_proba2[df_proba2['phage']=='p003pkpresabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[7.18093300e-01, 2.81887650e-01, 1.89898330e-05],
       [9.79965800e-01, 2.00342330e-02, 3.79279470e-10],
       [7.30386200e-02, 9.26956830e-01, 4.47887800e-06],
       [3.42183340e-04, 2.07277180e-03, 9.97585060e-01],
       [3.21410900e-12, 8.43131860e-04, 9.99156830e-01],
       [3.42183340e-04, 2.07277180e-03, 9.97585060e-01],
       [3.21410900e-12, 8.43131860e-04, 9.99156830e-01],
       [3.93137000e-04, 9.93089850e-01, 6.51700750e-03],
       [3.21410900e-12, 8.43131860e-04, 9.99156830e-01],
       [1.51658310e-04, 3.02637500e-02, 9.69584500e-01],
       [6.82273270e-01, 3.17697320e-01, 2.93398680e-05],
       [4.08698850e-03, 9.95903900e-01, 9.11266500e-06],
       [9.99567100e-01, 4.32776600e-04, 1.56440590e-07],
       [9.98988200e-01, 7.50830500e-04, 2.60972800e-04],
       [3.42183340e-04, 2.07277180e-03, 9.97585060e-01],
       [7.30386200e-02, 9.26956830e-01, 4.47887800e-06],
       [6.82273270e-01, 3.17697320e-01, 2.93398680e-05],
       [3.42183340e-04, 2.07277

In [120]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.98909767610748

In [121]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.98909767610748

In [122]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [123]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [124]:
dat3

,0,test
0,NRS148,2
1,NRS209,2
2,NRS187,1
3,CFBREBSa116,0
4,NRS187,1
...,...,...
197,GA12,0
198,NRS209,2
199,NRS265,1
200,NRS253,1


In [125]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [44]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 316us/step - loss: 8.1953 - accuracy: 0.5255 - val_loss: 4.4994 - val_accuracy: 0.4851
Epoch 2/100
470/470 [==============================] - 0s 193us/step - loss: 2.9493 - accuracy: 0.5106 - val_loss: 2.0852 - val_accuracy: 0.5198
Epoch 3/100
470/470 [==============================] - 0s 224us/step - loss: 1.2796 - accuracy: 0.5574 - val_loss: 1.1100 - val_accuracy: 0.6881
Epoch 4/100
470/470 [==============================] - 0s 221us/step - loss: 0.7187 - accuracy: 0.7362 - val_loss: 0.5912 - val_accuracy: 0.7327
Epoch 5/100
470/470 [==============================] - 0s 188us/step - loss: 0.5239 - accuracy: 0.8043 - val_loss: 0.5104 - val_accuracy: 0.8366
Epoch 6/100
470/470 [==============================] - 0s 223us/step - loss: 0.4743 - accuracy: 0.8213 - val_loss: 0.4821 - val_accuracy: 0.8119
Epoch 7/100
470/470 [==============================] - 0s 184us/step - loss: 0.4135 

Epoch 57/100
470/470 [==============================] - 0s 217us/step - loss: 0.2247 - accuracy: 0.9319 - val_loss: 0.7964 - val_accuracy: 0.8564
Epoch 58/100
470/470 [==============================] - 0s 153us/step - loss: 0.4386 - accuracy: 0.8809 - val_loss: 0.3433 - val_accuracy: 0.8663
Epoch 59/100
470/470 [==============================] - 0s 106us/step - loss: 0.3278 - accuracy: 0.8745 - val_loss: 0.1975 - val_accuracy: 0.9208
Epoch 60/100
470/470 [==============================] - 0s 113us/step - loss: 0.1635 - accuracy: 0.9447 - val_loss: 0.1986 - val_accuracy: 0.9307
Epoch 61/100
470/470 [==============================] - 0s 122us/step - loss: 0.1778 - accuracy: 0.9532 - val_loss: 0.1799 - val_accuracy: 0.9307
Epoch 62/100
470/470 [==============================] - 0s 369us/step - loss: 0.1657 - accuracy: 0.9532 - val_loss: 0.1975 - val_accuracy: 0.9307
Epoch 63/100
470/470 [==============================] - 0s 261us/step - loss: 0.1522 - accuracy: 0.9553 - val_loss: 0.1612 -

In [51]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

202/202 [==============================] - 0s 106us/step
over-sampling test accuracy: 96.53%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([2, 2, 1, 0, 1, 2, 2, 0, 2, 1, 1, 0, 2, 1, 0, 0, 0, 1, 2, 1, 0, 0,
       0, 0, 1, 0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 2, 0, 1, 0, 0, 1, 0, 1, 2,
       0, 1, 1, 0, 1, 0, 0, 2, 2, 1, 0, 1, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2,
       2, 2, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 2, 1, 1, 0, 1, 2, 1, 1, 1, 1,
       2, 2, 2, 1, 0, 1, 0, 1, 0, 2, 0, 1, 2, 2, 0, 2, 1, 2, 0, 1, 0, 0,
       0, 2, 1, 0, 0, 2, 1, 1, 0, 0, 0, 1, 0, 1, 1, 2, 1, 2, 0, 2, 0, 1,
       0, 1, 2, 2, 0, 0, 1, 2, 2, 2, 0, 2, 2, 2, 0, 0, 1, 0, 2, 1, 2, 0,
       0, 2, 1, 1, 1, 2, 1, 0, 0, 2, 2, 0, 1, 2, 2, 0, 2, 1, 0, 0, 2, 1,
       1, 2, 0, 0, 0, 1, 1, 2, 1, 2, 0, 2, 2, 2, 0, 0, 1, 2, 0, 1, 1, 0,
       2, 1, 1, 0])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,NRS148,2,2
1,NRS209,2,2
2,NRS187,1,1
3,CFBREBSa116,0,0
4,NRS187,1,1
...,...,...,...
197,GA12,0,0
198,NRS209,2,2
199,NRS265,1,1
200,NRS253,1,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,3.163079e-04,0.073239,9.264445e-01
1,4.834474e-11,0.001598,9.984019e-01
2,1.197055e-01,0.880255,3.925587e-05
3,8.983797e-01,0.101562,5.845739e-05
4,1.197055e-01,0.880255,3.925587e-05
...,...,...,...
197,9.921768e-01,0.007823,2.947980e-11
198,4.834474e-11,0.001598,9.984019e-01
199,3.676001e-02,0.950768,1.247150e-02
200,5.030434e-03,0.994785,1.842785e-04


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p003ST.csv", index = False,
         header=None)

In [55]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 189us/step - loss: 0.1285 - accuracy: 0.9660 - val_loss: 0.1325 - val_accuracy: 0.9604
Epoch 2/100
470/470 [==============================] - 0s 193us/step - loss: 0.1009 - accuracy: 0.9681 - val_loss: 0.1184 - val_accuracy: 0.9604
Epoch 3/100
470/470 [==============================] - 0s 192us/step - loss: 0.1045 - accuracy: 0.9596 - val_loss: 0.1087 - val_accuracy: 0.9554
Epoch 4/100
470/470 [==============================] - 0s 176us/step - loss: 0.0998 - accuracy: 0.9681 - val_loss: 0.1818 - val_accuracy: 0.9059
Epoch 5/100
470/470 [==============================] - 0s 155us/step - loss: 0.1190 - accuracy: 0.9574 - val_loss: 0.1162 - val_accuracy: 0.9604
Epoch 6/100
470/470 [==============================] - 0s 147us/step - loss: 0.1448 - accuracy: 0.9468 - val_loss: 0.1519 - val_accuracy: 0.9604
Epoch 7/100
470/470 [==============================] - 0s 124us/step - loss: 0.1518 

Epoch 57/100
470/470 [==============================] - 0s 214us/step - loss: 0.1056 - accuracy: 0.9574 - val_loss: 0.1197 - val_accuracy: 0.9604
Epoch 58/100
470/470 [==============================] - 0s 236us/step - loss: 0.0975 - accuracy: 0.9511 - val_loss: 0.1192 - val_accuracy: 0.9455
Epoch 59/100
470/470 [==============================] - 0s 143us/step - loss: 0.0977 - accuracy: 0.9702 - val_loss: 0.1028 - val_accuracy: 0.9554
Epoch 60/100
470/470 [==============================] - 0s 143us/step - loss: 0.0884 - accuracy: 0.9702 - val_loss: 0.0994 - val_accuracy: 0.9604
Epoch 61/100
470/470 [==============================] - 0s 175us/step - loss: 0.0872 - accuracy: 0.9638 - val_loss: 0.1264 - val_accuracy: 0.9604
Epoch 62/100
470/470 [==============================] - 0s 153us/step - loss: 0.1011 - accuracy: 0.9638 - val_loss: 0.1021 - val_accuracy: 0.9604
Epoch 63/100
470/470 [==============================] - 0s 182us/step - loss: 0.0873 - accuracy: 0.9723 - val_loss: 0.0989 -

In [56]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 96.19%


In [126]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [127]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [128]:
y_prob3 = df_proba3[df_proba3['phage']=='p003pkpresabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[3.16307940e-04, 7.32391300e-02, 9.26444530e-01],
       [4.83447400e-11, 1.59805550e-03, 9.98401940e-01],
       [1.19705480e-01, 8.80255200e-01, 3.92558720e-05],
       [8.98379740e-01, 1.01561815e-01, 5.84573940e-05],
       [1.19705480e-01, 8.80255200e-01, 3.92558720e-05],
       [4.83447400e-11, 1.59805550e-03, 9.98401940e-01],
       [1.75080610e-03, 2.00983560e-03, 9.96239300e-01],
       [8.98379740e-01, 1.01561815e-01, 5.84573940e-05],
       [3.16307940e-04, 7.32391300e-02, 9.26444530e-01],
       [3.67600060e-02, 9.50768400e-01, 1.24715045e-02],
       [6.69607900e-02, 9.32511300e-01, 5.27940400e-04],
       [9.99560060e-01, 4.39986350e-04, 3.78957450e-10],
       [4.83447400e-11, 1.59805550e-03, 9.98401940e-01],
       [2.25106100e-01, 7.74045470e-01, 8.48337130e-04],
       [9.82213740e-01, 1.77863390e-02, 2.56931130e-11],
       [8.98379740e-01, 1.01561815e-01, 5.84573940e-05],
       [9.96455200e-01, 3.54470520e-03, 1.26692410e-07],
       [1.32752250e-02, 9.85978

In [129]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9936111924039931

In [130]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9936111924039931

In [131]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [132]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [133]:
dat4

,0,test
0,CFBRSa04,0
1,NRS021,0
2,NRS073,0
3,NRS049,0
4,CA541,1
...,...,...
197,NRS387,1
198,SR1746,0
199,NRS148,2
200,NRS255,2


In [134]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [57]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 327us/step - loss: 3.9472 - accuracy: 0.5255 - val_loss: 4.6515 - val_accuracy: 0.5297
Epoch 2/100
470/470 [==============================] - 0s 140us/step - loss: 1.7468 - accuracy: 0.6085 - val_loss: 2.3568 - val_accuracy: 0.4851
Epoch 3/100
470/470 [==============================] - 0s 200us/step - loss: 1.0450 - accuracy: 0.7085 - val_loss: 0.8285 - val_accuracy: 0.7673
Epoch 4/100
470/470 [==============================] - 0s 222us/step - loss: 0.5841 - accuracy: 0.7787 - val_loss: 0.4784 - val_accuracy: 0.7871
Epoch 5/100
470/470 [==============================] - 0s 187us/step - loss: 0.5503 - accuracy: 0.7872 - val_loss: 0.5056 - val_accuracy: 0.8614
Epoch 6/100
470/470 [==============================] - 0s 195us/step - loss: 0.4720 - accuracy: 0.8191 - val_loss: 0.5152 - val_accuracy: 0.7970
Epoch 7/100
470/470 [==============================] - 0s 187us/step - loss: 0.4242 

Epoch 57/100
470/470 [==============================] - 0s 109us/step - loss: 0.1557 - accuracy: 0.9532 - val_loss: 0.1397 - val_accuracy: 0.9554
Epoch 58/100
470/470 [==============================] - 0s 102us/step - loss: 0.1885 - accuracy: 0.9404 - val_loss: 0.1294 - val_accuracy: 0.9703
Epoch 59/100
470/470 [==============================] - 0s 139us/step - loss: 0.1886 - accuracy: 0.9213 - val_loss: 0.1371 - val_accuracy: 0.9307
Epoch 60/100
470/470 [==============================] - 0s 306us/step - loss: 0.1700 - accuracy: 0.9404 - val_loss: 0.1525 - val_accuracy: 0.9208
Epoch 61/100
470/470 [==============================] - 0s 293us/step - loss: 0.1967 - accuracy: 0.9234 - val_loss: 0.1940 - val_accuracy: 0.8861
Epoch 62/100
470/470 [==============================] - 0s 149us/step - loss: 0.1818 - accuracy: 0.9362 - val_loss: 0.1746 - val_accuracy: 0.9059
Epoch 63/100
470/470 [==============================] - 0s 111us/step - loss: 0.1629 - accuracy: 0.9426 - val_loss: 0.1394 -

In [64]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

202/202 [==============================] - 0s 115us/step
over-sampling test accuracy: 96.53%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 1, 0, 2, 1, 2, 0,
       0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 2, 0, 2, 2, 1, 1, 1, 1, 1, 2, 0, 0,
       0, 1, 2, 1, 1, 0, 1, 2, 0, 1, 2, 0, 1, 1, 2, 0, 0, 2, 2, 2, 0, 0,
       2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 0, 1, 0, 1, 1, 0, 2, 2, 0, 2,
       2, 0, 2, 0, 0, 2, 1, 0, 2, 2, 2, 1, 1, 2, 0, 2, 1, 1, 1, 2, 0, 0,
       0, 1, 2, 1, 1, 2, 0, 0, 2, 0, 0, 2, 1, 2, 1, 2, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 2, 1, 0, 0, 1, 1, 2, 2, 2, 2,
       1, 2, 1, 2, 0, 2, 0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 0, 2, 1,
       2, 2, 1, 2, 1, 2, 2, 0, 0, 2, 2, 0, 0, 2, 1, 0, 0, 2, 2, 2, 0, 1,
       0, 2, 2, 1])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,CFBRSa04,0,0
1,NRS021,0,0
2,NRS073,0,0
3,NRS049,0,0
4,CA541,1,0
...,...,...,...
197,NRS387,1,1
198,SR1746,0,0
199,NRS148,2,2
200,NRS255,2,2


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,0.948561,0.051417,2.209942e-05
1,0.996933,0.003068,5.170221e-09
2,0.996933,0.003068,5.170221e-09
3,0.894868,0.105105,2.778030e-05
4,0.894868,0.105105,2.778030e-05
...,...,...,...
197,0.241478,0.758325,1.975381e-04
198,0.976457,0.023543,5.295739e-08
199,0.004439,0.160147,8.354138e-01
200,0.002211,0.000872,9.969168e-01


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p003ST.csv", index = False,
         header=None)

In [68]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 199us/step - loss: 0.1073 - accuracy: 0.9681 - val_loss: 0.0934 - val_accuracy: 0.9653
Epoch 2/100
470/470 [==============================] - 0s 196us/step - loss: 0.1034 - accuracy: 0.9681 - val_loss: 0.0927 - val_accuracy: 0.9703
Epoch 3/100
470/470 [==============================] - 0s 192us/step - loss: 0.1026 - accuracy: 0.9638 - val_loss: 0.0936 - val_accuracy: 0.9703
Epoch 4/100
470/470 [==============================] - 0s 177us/step - loss: 0.1047 - accuracy: 0.9638 - val_loss: 0.0876 - val_accuracy: 0.9653
Epoch 5/100
470/470 [==============================] - 0s 166us/step - loss: 0.1071 - accuracy: 0.9681 - val_loss: 0.0862 - val_accuracy: 0.9653
Epoch 6/100
470/470 [==============================] - 0s 125us/step - loss: 0.1035 - accuracy: 0.9596 - val_loss: 0.0918 - val_accuracy: 0.9703
Epoch 7/100
470/470 [==============================] - 0s 109us/step - loss: 0.1034 

Epoch 57/100
470/470 [==============================] - 0s 117us/step - loss: 0.0852 - accuracy: 0.9681 - val_loss: 0.0784 - val_accuracy: 0.9703
Epoch 58/100
470/470 [==============================] - 0s 108us/step - loss: 0.0872 - accuracy: 0.9681 - val_loss: 0.0825 - val_accuracy: 0.9653
Epoch 59/100
470/470 [==============================] - 0s 108us/step - loss: 0.0865 - accuracy: 0.9660 - val_loss: 0.0788 - val_accuracy: 0.9703
Epoch 60/100
470/470 [==============================] - 0s 113us/step - loss: 0.0895 - accuracy: 0.9681 - val_loss: 0.0788 - val_accuracy: 0.9653
Epoch 61/100
470/470 [==============================] - 0s 213us/step - loss: 0.0853 - accuracy: 0.9660 - val_loss: 0.0780 - val_accuracy: 0.9703
Epoch 62/100
470/470 [==============================] - 0s 179us/step - loss: 0.0861 - accuracy: 0.9660 - val_loss: 0.0801 - val_accuracy: 0.9653
Epoch 63/100
470/470 [==============================] - 0s 383us/step - loss: 0.0998 - accuracy: 0.9660 - val_loss: 0.0918 -

In [69]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 96.57%


In [135]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [136]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [137]:
y_prob4 = df_proba4[df_proba4['phage']=='p003pkpresabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[9.48561400e-01, 5.14166360e-02, 2.20994170e-05],
       [9.96932500e-01, 3.06751320e-03, 5.17022100e-09],
       [9.96932500e-01, 3.06751320e-03, 5.17022100e-09],
       [8.94867540e-01, 1.05104730e-01, 2.77802960e-05],
       [8.94867540e-01, 1.05104730e-01, 2.77802960e-05],
       [6.50246800e-02, 9.34975300e-01, 7.61967100e-09],
       [1.66112720e-02, 9.82741530e-01, 6.47174300e-04],
       [1.48616920e-01, 8.51296600e-01, 8.64549950e-05],
       [8.94867540e-01, 1.05104730e-01, 2.77802960e-05],
       [9.59392800e-01, 4.05865940e-02, 2.05507040e-05],
       [9.35723300e-01, 6.42458800e-02, 3.07378650e-05],
       [7.47551560e-01, 2.52355580e-01, 9.28358400e-05],
       [2.21077980e-03, 8.72457050e-04, 9.96916800e-01],
       [9.70485900e-10, 2.90616530e-04, 9.99709300e-01],
       [2.21077980e-03, 8.72457050e-04, 9.96916800e-01],
       [9.70485900e-10, 2.90616530e-04, 9.99709300e-01],
       [6.90548630e-03, 9.92323040e-01, 7.71391960e-04],
       [9.93783700e-01, 6.21628

In [138]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.993033200013007

In [139]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.993033200013007

In [140]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9903976509066866

In [141]:
np.std(ovos)

0.003148704381461307

In [142]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9903976509066866

In [143]:
np.std(ovrs)

0.003148704381461307

In [70]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [71]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 94.68%


In [72]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.022242804066183213


In [73]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [74]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 96.46%


In [75]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.0032066016


In [144]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_over)

In [145]:
dat5 = pd.DataFrame(X_test_over[:,0])
dat5['test'] = y_test_over

In [146]:
dat5

,0,test
0,NRS255,2
1,NRS148,2
2,NRS209,2
3,NRS386,1
4,NRS230,0
...,...,...
197,NRS209,2
198,NRS209,2
199,SR4153,0
200,NRS255,2


In [147]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [56]:
#### add regularizer and dropout
model1_over5 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [71]:
model1_over5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [72]:
model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 716us/step - loss: 8.6878 - accuracy: 0.4979 - val_loss: 5.1503 - val_accuracy: 0.5693
Epoch 2/100
470/470 [==============================] - 0s 191us/step - loss: 6.9972 - accuracy: 0.5787 - val_loss: 4.0346 - val_accuracy: 0.6089
Epoch 3/100
470/470 [==============================] - 0s 222us/step - loss: 6.4895 - accuracy: 0.5064 - val_loss: 3.1314 - val_accuracy: 0.5792
Epoch 4/100
470/470 [==============================] - 0s 229us/step - loss: 4.9844 - accuracy: 0.5553 - val_loss: 2.1448 - val_accuracy: 0.6485
Epoch 5/100
470/470 [==============================] - 0s 143us/step - loss: 4.4914 - accuracy: 0.5596 - val_loss: 1.7362 - val_accuracy: 0.6881
Epoch 6/100
470/470 [==============================] - 0s 203us/step - loss: 3.8642 - accuracy: 0.5957 - val_loss: 1.4299 - val_accuracy: 0.7030
Epoch 7/100
470/470 [==============================] - 0s 288us/step - loss: 4.1442 

Epoch 57/100
470/470 [==============================] - 0s 143us/step - loss: 1.9922 - accuracy: 0.7234 - val_loss: 1.3804 - val_accuracy: 0.8515
Epoch 58/100
470/470 [==============================] - 0s 113us/step - loss: 1.7927 - accuracy: 0.7489 - val_loss: 1.0250 - val_accuracy: 0.8762
Epoch 59/100
470/470 [==============================] - 0s 121us/step - loss: 1.9099 - accuracy: 0.7340 - val_loss: 1.0496 - val_accuracy: 0.8713
Epoch 60/100
470/470 [==============================] - 0s 176us/step - loss: 1.6334 - accuracy: 0.7511 - val_loss: 0.9007 - val_accuracy: 0.8713
Epoch 61/100
470/470 [==============================] - 0s 132us/step - loss: 1.6199 - accuracy: 0.7766 - val_loss: 1.0265 - val_accuracy: 0.8713
Epoch 62/100
470/470 [==============================] - 0s 119us/step - loss: 1.8629 - accuracy: 0.7128 - val_loss: 1.0597 - val_accuracy: 0.8762
Epoch 63/100
470/470 [==============================] - 0s 115us/step - loss: 1.7487 - accuracy: 0.7277 - val_loss: 0.9719 -

In [83]:
acc_test_over5 = model1_over5.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over5*100))

202/202 [==============================] - 0s 428us/step
over-sampling test accuracy: 88.12%


In [73]:
pred5 = model1_over5.predict_classes(X_test_over)
pred5

array([2, 2, 2, 1, 0, 0, 0, 1, 2, 0, 1, 0, 2, 0, 1, 0, 0, 2, 0, 1, 1, 2,
       2, 2, 1, 0, 0, 1, 2, 1, 2, 1, 1, 2, 1, 2, 0, 2, 0, 1, 1, 2, 2, 0,
       2, 2, 2, 1, 0, 0, 0, 0, 0, 0, 2, 1, 2, 2, 1, 1, 2, 2, 0, 0, 1, 0,
       1, 2, 1, 0, 2, 0, 1, 2, 0, 2, 1, 2, 1, 2, 2, 0, 1, 2, 2, 2, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 2, 0, 2, 1, 2, 0, 0, 1, 0, 1, 2, 1, 1, 2, 0,
       0, 2, 1, 0, 1, 2, 0, 2, 1, 2, 0, 1, 0, 2, 0, 0, 2, 0, 1, 0, 1, 1,
       2, 2, 0, 0, 1, 1, 0, 2, 1, 2, 1, 1, 0, 2, 1, 1, 1, 1, 0, 0, 0, 2,
       0, 0, 0, 1, 2, 0, 2, 0, 0, 2, 2, 1, 1, 1, 0, 2, 0, 0, 0, 0, 2, 1,
       1, 0, 1, 0, 2, 0, 2, 0, 1, 0, 0, 1, 2, 1, 1, 2, 2, 2, 2, 0, 2, 2,
       2, 0, 2, 2])

In [74]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,NRS255,2,2
1,NRS148,2,2
2,NRS209,2,2
3,NRS386,1,1
4,NRS230,0,0
...,...,...,...
197,NRS209,2,2
198,NRS209,2,2
199,SR4153,0,0
200,NRS255,2,2


In [75]:
proba5 = model1_over5.predict_proba(X_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [76]:
dat_proba5

,0,1,2
0,1.258997e-04,2.146967e-03,9.977271e-01
1,2.622584e-17,1.171789e-14,1.000000e+00
2,1.312901e-12,1.454831e-10,1.000000e+00
3,2.127909e-02,9.786837e-01,3.726870e-05
4,1.000000e+00,2.345151e-08,1.853019e-17
...,...,...,...
197,1.312901e-12,1.454831e-10,1.000000e+00
198,1.312901e-12,1.454831e-10,1.000000e+00
199,1.000000e+00,0.000000e+00,0.000000e+00
200,1.258997e-04,2.146967e-03,9.977271e-01


In [77]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [78]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p003ST.csv", index = False,
         header=None)

In [87]:
hist1_over5 = model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 229us/step - loss: 1.4322 - accuracy: 0.7277 - val_loss: 0.7450 - val_accuracy: 0.8911
Epoch 2/100
470/470 [==============================] - 0s 219us/step - loss: 1.6145 - accuracy: 0.7340 - val_loss: 0.6549 - val_accuracy: 0.8812
Epoch 3/100
470/470 [==============================] - 0s 193us/step - loss: 1.4263 - accuracy: 0.7383 - val_loss: 0.7513 - val_accuracy: 0.8812
Epoch 4/100
470/470 [==============================] - 0s 213us/step - loss: 1.3111 - accuracy: 0.7447 - val_loss: 0.8308 - val_accuracy: 0.8416
Epoch 5/100
470/470 [==============================] - 0s 186us/step - loss: 1.3749 - accuracy: 0.7191 - val_loss: 0.5903 - val_accuracy: 0.9208
Epoch 6/100
470/470 [==============================] - 0s 189us/step - loss: 1.5134 - accuracy: 0.7255 - val_loss: 0.6759 - val_accuracy: 0.8911
Epoch 7/100
470/470 [==============================] - 0s 198us/step - loss: 1.4627 

Epoch 57/100
470/470 [==============================] - 0s 163us/step - loss: 1.0936 - accuracy: 0.7383 - val_loss: 0.4372 - val_accuracy: 0.9505
Epoch 58/100
470/470 [==============================] - 0s 145us/step - loss: 0.8967 - accuracy: 0.7957 - val_loss: 0.4540 - val_accuracy: 0.9455
Epoch 59/100
470/470 [==============================] - 0s 113us/step - loss: 0.9847 - accuracy: 0.7809 - val_loss: 0.4544 - val_accuracy: 0.9604
Epoch 60/100
470/470 [==============================] - 0s 116us/step - loss: 0.9074 - accuracy: 0.7830 - val_loss: 0.4679 - val_accuracy: 0.9257
Epoch 61/100
470/470 [==============================] - 0s 110us/step - loss: 1.0340 - accuracy: 0.7596 - val_loss: 0.7131 - val_accuracy: 0.9307
Epoch 62/100
470/470 [==============================] - 0s 123us/step - loss: 1.0471 - accuracy: 0.7574 - val_loss: 0.7239 - val_accuracy: 0.9307
Epoch 63/100
470/470 [==============================] - 0s 114us/step - loss: 1.1341 - accuracy: 0.7532 - val_loss: 0.7166 -

In [88]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over5.history['accuracy'])*100))

over-sampling train accuracy: 75.76%


In [148]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [149]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS241,1,1,1.342914e-03,9.986569e-01,2.348628e-07
1,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
2,p002ykpresabsSTCC_qual,NRS255,1,1,1.780311e-07,9.999999e-01,2.544841e-12
3,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,2.203547e-10,5.688883e-15
4,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,BCH-SA-12,0,0,1.000000e+00,1.152503e-09,1.898730e-09
1978,pyopresabsSTCC_qual,NRS049,0,1,8.401357e-11,1.000000e+00,3.209735e-13
1979,pyopresabsSTCC_qual,NRS022,0,0,1.000000e+00,4.755084e-10,1.974275e-10
1980,pyopresabsSTCC_qual,NRS236,1,1,1.357345e-08,1.000000e+00,1.293117e-10


In [150]:
y_prob5 = df_proba5[df_proba5['phage']=='p003pkpresabsSTCC_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[1.25899660e-04, 2.14696680e-03, 9.97727100e-01],
       [2.62258360e-17, 1.17178900e-14, 1.00000000e+00],
       [1.31290120e-12, 1.45483140e-10, 1.00000000e+00],
       [2.12790900e-02, 9.78683650e-01, 3.72687030e-05],
       [1.00000000e+00, 2.34515150e-08, 1.85301940e-17],
       [1.00000000e+00, 4.58270300e-08, 1.02282040e-15],
       [9.99998200e-01, 1.73866520e-06, 1.12610840e-17],
       [1.69548460e-07, 9.99999900e-01, 1.27188560e-09],
       [1.31290120e-12, 1.45483140e-10, 1.00000000e+00],
       [9.81584800e-01, 1.84152050e-02, 3.88670760e-13],
       [1.69548460e-07, 9.99999900e-01, 1.27188560e-09],
       [9.81584800e-01, 1.84152050e-02, 3.88670760e-13],
       [1.25899660e-04, 2.14696680e-03, 9.97727100e-01],
       [1.00000000e+00, 9.93071940e-12, 3.48034750e-21],
       [1.28087640e-01, 8.71086660e-01, 8.25726140e-04],
       [9.81584800e-01, 1.84152050e-02, 3.88670760e-13],
       [1.00000000e+00, 6.42052260e-14, 1.62811880e-18],
       [1.31290120e-12, 1.45483

In [151]:
ovo5 = rocauc_ovo(y_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.967152120660315

In [152]:
ovr5 = rocauc_ovr(y_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.967152120660315

In [153]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_over)

In [154]:
dat6 = pd.DataFrame(X_test_over[:,0])
dat6['test'] = y_test_over

In [155]:
dat6

,0,test
0,115,1
1,NRS209,2
2,GA984,0
3,NRS187,1
4,NRS148,2
...,...,...
197,NRS253,1
198,EUH15,0
199,NRS180,1
200,NRS266,1


In [156]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [83]:
model1_over6 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [84]:
model1_over6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 743us/step - loss: 9.9542 - accuracy: 0.3745 - val_loss: 5.8433 - val_accuracy: 0.4703
Epoch 2/100
470/470 [==============================] - 0s 244us/step - loss: 8.1247 - accuracy: 0.4213 - val_loss: 4.7025 - val_accuracy: 0.4505
Epoch 3/100
470/470 [==============================] - 0s 199us/step - loss: 6.1863 - accuracy: 0.4489 - val_loss: 2.2844 - val_accuracy: 0.5495
Epoch 4/100
470/470 [==============================] - 0s 243us/step - loss: 5.0407 - accuracy: 0.4915 - val_loss: 0.9682 - val_accuracy: 0.6139
Epoch 5/100
470/470 [==============================] - 0s 274us/step - loss: 3.3077 - accuracy: 0.5532 - val_loss: 0.8372 - val_accuracy: 0.6139
Epoch 6/100
470/470 [==============================] - 0s 143us/step - loss: 3.1675 - accuracy: 0.5872 - val_loss: 0.7498 - val_accuracy: 0.6535
Epoch 7/100
470/470 [==============================] - 0s 440us/step - loss: 3.0265 

Epoch 57/100
470/470 [==============================] - 0s 181us/step - loss: 2.3475 - accuracy: 0.6574 - val_loss: 0.7120 - val_accuracy: 0.8564
Epoch 58/100
470/470 [==============================] - 0s 138us/step - loss: 2.1451 - accuracy: 0.6894 - val_loss: 0.8748 - val_accuracy: 0.8960
Epoch 59/100
470/470 [==============================] - 0s 122us/step - loss: 2.0915 - accuracy: 0.7213 - val_loss: 0.8936 - val_accuracy: 0.8861
Epoch 60/100
470/470 [==============================] - 0s 116us/step - loss: 1.9944 - accuracy: 0.7149 - val_loss: 0.8410 - val_accuracy: 0.9010
Epoch 61/100
470/470 [==============================] - 0s 123us/step - loss: 1.8754 - accuracy: 0.7021 - val_loss: 0.9756 - val_accuracy: 0.8515
Epoch 62/100
470/470 [==============================] - 0s 118us/step - loss: 2.1572 - accuracy: 0.6638 - val_loss: 0.7440 - val_accuracy: 0.9010
Epoch 63/100
470/470 [==============================] - 0s 125us/step - loss: 1.9212 - accuracy: 0.6936 - val_loss: 0.8766 -

In [96]:
acc_test_over6 = model1_over6.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over6*100))

202/202 [==============================] - 0s 108us/step
over-sampling test accuracy: 92.08%


In [86]:
pred6 = model1_over6.predict_classes(X_test_over)
pred6

array([1, 2, 1, 1, 2, 2, 1, 1, 1, 0, 2, 1, 2, 1, 0, 0, 0, 2, 1, 1, 0, 1,
       2, 0, 2, 1, 2, 0, 0, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 0, 1,
       2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 0, 1, 0, 1, 0, 1, 1, 1, 2, 1,
       1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 0, 2, 2, 0, 2, 2, 2, 1, 1, 1, 1,
       0, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2, 1, 0, 0, 0, 2, 2, 0, 2, 1, 2, 2,
       1, 1, 0, 2, 1, 0, 1, 1, 2, 1, 2, 2, 1, 1, 0, 0, 2, 1, 1, 1, 1, 2,
       1, 0, 2, 2, 1, 2, 0, 1, 2, 0, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1,
       1, 2, 0, 2, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 0, 2, 2,
       1, 1, 2, 2, 2, 0, 0, 1, 1, 0, 0, 0, 2, 2, 1, 0, 1, 1, 1, 0, 2, 1,
       0, 1, 1, 1])

In [87]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,115,1,1
1,NRS209,2,2
2,GA984,0,1
3,NRS187,1,1
4,NRS148,2,2
...,...,...,...
197,NRS253,1,1
198,EUH15,0,0
199,NRS180,1,1
200,NRS266,1,1


In [88]:
proba6 = model1_over6.predict_proba(X_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [89]:
dat_proba6

,0,1,2
0,3.378712e-08,1.000000e+00,2.580147e-21
1,5.755056e-10,1.493326e-09,1.000000e+00
2,3.207942e-01,6.746280e-01,4.577814e-03
3,6.123706e-06,9.999939e-01,2.119516e-15
4,8.780598e-11,2.499853e-10,1.000000e+00
...,...,...,...
197,3.207942e-01,6.746280e-01,4.577814e-03
198,9.992932e-01,7.067574e-04,3.600648e-14
199,4.149153e-09,1.000000e+00,9.871512e-24
200,1.619248e-08,1.000000e+00,3.663113e-22


In [90]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [91]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p003ST.csv", index = False,
         header=None)

In [100]:
hist1_over6 = model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 231us/step - loss: 1.9564 - accuracy: 0.7383 - val_loss: 0.7850 - val_accuracy: 0.9455
Epoch 2/100
470/470 [==============================] - 0s 165us/step - loss: 2.0272 - accuracy: 0.7362 - val_loss: 0.6557 - val_accuracy: 0.9455
Epoch 3/100
470/470 [==============================] - 0s 135us/step - loss: 1.5909 - accuracy: 0.7255 - val_loss: 0.8612 - val_accuracy: 0.8416
Epoch 4/100
470/470 [==============================] - 0s 125us/step - loss: 1.9322 - accuracy: 0.7660 - val_loss: 0.8010 - val_accuracy: 0.9356
Epoch 5/100
470/470 [==============================] - 0s 121us/step - loss: 2.1065 - accuracy: 0.7383 - val_loss: 0.8736 - val_accuracy: 0.9307
Epoch 6/100
470/470 [==============================] - 0s 130us/step - loss: 1.9895 - accuracy: 0.7447 - val_loss: 0.6777 - val_accuracy: 0.8911
Epoch 7/100
470/470 [==============================] - 0s 119us/step - loss: 1.8077 

Epoch 57/100
470/470 [==============================] - 0s 219us/step - loss: 1.4055 - accuracy: 0.7617 - val_loss: 0.9447 - val_accuracy: 0.8663
Epoch 58/100
470/470 [==============================] - 0s 282us/step - loss: 1.8504 - accuracy: 0.7149 - val_loss: 0.7486 - val_accuracy: 0.9406
Epoch 59/100
470/470 [==============================] - 0s 214us/step - loss: 1.2528 - accuracy: 0.7915 - val_loss: 0.7591 - val_accuracy: 0.9208
Epoch 60/100
470/470 [==============================] - 0s 217us/step - loss: 1.5172 - accuracy: 0.7617 - val_loss: 0.7406 - val_accuracy: 0.9455
Epoch 61/100
470/470 [==============================] - 0s 218us/step - loss: 1.5605 - accuracy: 0.7660 - val_loss: 0.7458 - val_accuracy: 0.9406
Epoch 62/100
470/470 [==============================] - 0s 205us/step - loss: 1.5783 - accuracy: 0.7489 - val_loss: 0.7738 - val_accuracy: 0.9505
Epoch 63/100
470/470 [==============================] - 0s 237us/step - loss: 1.5278 - accuracy: 0.7638 - val_loss: 0.7930 -

In [101]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over6.history['accuracy'])*100))

over-sampling train accuracy: 75.31%


In [157]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [158]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,1.790400e-08,4.141849e-08,1.000000e+00
1,p002ykpresabsSTCC_qual,NRS386,1,1,5.739934e-04,9.994259e-01,6.773014e-08
2,p002ykpresabsSTCC_qual,NRS148,2,2,5.286934e-09,1.269109e-08,1.000000e+00
3,p002ykpresabsSTCC_qual,NRS178,0,1,6.494936e-12,1.000000e+00,2.537080e-25
4,p002ykpresabsSTCC_qual,NRS237,0,1,5.701098e-02,9.399204e-01,3.068583e-03
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS272,0,0,9.999607e-01,3.367024e-05,5.776848e-06
1978,pyopresabsSTCC_qual,NRS112,1,1,8.275442e-08,9.999999e-01,3.739556e-09
1979,pyopresabsSTCC_qual,NRS064,1,1,2.168245e-08,1.000000e+00,9.603962e-09
1980,pyopresabsSTCC_qual,BCH-SA-04,0,0,1.000000e+00,1.026408e-15,1.630406e-14


In [159]:
y_prob6 = df_proba6[df_proba6['phage']=='p003pkpresabsSTCC_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[3.37871240e-08, 1.00000000e+00, 2.58014660e-21],
       [5.75505640e-10, 1.49332610e-09, 1.00000000e+00],
       [3.20794200e-01, 6.74628000e-01, 4.57781370e-03],
       [6.12370650e-06, 9.99993900e-01, 2.11951640e-15],
       [8.78059850e-11, 2.49985250e-10, 1.00000000e+00],
       [5.75505640e-10, 1.49332610e-09, 1.00000000e+00],
       [6.76214640e-08, 9.99999900e-01, 3.30464290e-21],
       [1.51447530e-05, 9.99984860e-01, 2.15396800e-14],
       [4.05319240e-09, 1.00000000e+00, 9.27707100e-24],
       [9.96394100e-01, 3.60587050e-03, 1.22037580e-13],
       [5.75505640e-10, 1.49332610e-09, 1.00000000e+00],
       [5.47757430e-08, 1.00000000e+00, 9.30167800e-21],
       [5.75505640e-10, 1.49332610e-09, 1.00000000e+00],
       [1.51447530e-05, 9.99984860e-01, 2.15396800e-14],
       [9.99876000e-01, 1.23928620e-04, 1.61447960e-09],
       [9.99999640e-01, 3.89809430e-07, 2.33771300e-17],
       [9.99857200e-01, 1.42834440e-04, 2.34590540e-14],
       [5.75505640e-10, 1.49332

In [160]:
ovo6 = rocauc_ovo(y_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.9769854431545975

In [161]:
ovr6 = rocauc_ovr(y_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.9769854431545975

In [162]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_over)

In [163]:
dat7 = pd.DataFrame(X_test_over[:,0])
dat7['test'] = y_test_over

In [164]:
dat7

,0,test
0,NRS260,0
1,NRS148,2
2,NRS205,1
3,NRS064,1
4,NRS209,2
...,...,...
197,NRS255,2
198,GA53649,0
199,NRS209,2
200,NRS210,0


In [165]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [96]:
model1_over7 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [97]:
model1_over7.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 682us/step - loss: 10.9208 - accuracy: 0.3660 - val_loss: 6.7597 - val_accuracy: 0.5000
Epoch 2/100
470/470 [==============================] - 0s 227us/step - loss: 7.8083 - accuracy: 0.5149 - val_loss: 4.9314 - val_accuracy: 0.5396
Epoch 3/100
470/470 [==============================] - 0s 212us/step - loss: 6.2548 - accuracy: 0.4660 - val_loss: 3.3329 - val_accuracy: 0.5990
Epoch 4/100
470/470 [==============================] - 0s 181us/step - loss: 5.2616 - accuracy: 0.4638 - val_loss: 2.1933 - val_accuracy: 0.5693
Epoch 5/100
470/470 [==============================] - 0s 288us/step - loss: 4.1984 - accuracy: 0.5255 - val_loss: 1.5212 - val_accuracy: 0.6386
Epoch 6/100
470/470 [==============================] - 0s 215us/step - loss: 3.9139 - accuracy: 0.5170 - val_loss: 1.0769 - val_accuracy: 0.5941
Epoch 7/100
470/470 [==============================] - 0s 240us/step - loss: 3.7003

Epoch 57/100
470/470 [==============================] - 0s 155us/step - loss: 2.7128 - accuracy: 0.6574 - val_loss: 0.8765 - val_accuracy: 0.7475
Epoch 58/100
470/470 [==============================] - 0s 139us/step - loss: 2.4628 - accuracy: 0.6596 - val_loss: 1.0409 - val_accuracy: 0.8020
Epoch 59/100
470/470 [==============================] - 0s 132us/step - loss: 2.6706 - accuracy: 0.6872 - val_loss: 0.8989 - val_accuracy: 0.8168
Epoch 60/100
470/470 [==============================] - 0s 198us/step - loss: 2.4073 - accuracy: 0.6872 - val_loss: 0.9465 - val_accuracy: 0.8465
Epoch 61/100
470/470 [==============================] - 0s 184us/step - loss: 1.9870 - accuracy: 0.7213 - val_loss: 0.7696 - val_accuracy: 0.8366
Epoch 62/100
470/470 [==============================] - 0s 146us/step - loss: 2.2174 - accuracy: 0.6723 - val_loss: 0.7958 - val_accuracy: 0.8465
Epoch 63/100
470/470 [==============================] - 0s 190us/step - loss: 1.7835 - accuracy: 0.7553 - val_loss: 1.0330 -

In [109]:
acc_test_over7 = model1_over7.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over7*100))

202/202 [==============================] - 0s 123us/step
over-sampling test accuracy: 91.09%


In [99]:
pred7 = model1_over7.predict_classes(X_test_over)
pred7

array([1, 2, 2, 2, 2, 1, 0, 1, 2, 0, 0, 1, 2, 1, 2, 0, 2, 0, 1, 0, 0, 1,
       2, 0, 1, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 1, 1, 1, 1, 0, 2, 0, 1, 1,
       0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 1, 2, 2, 1, 0, 0, 2, 0, 1,
       2, 2, 2, 0, 2, 1, 2, 2, 1, 0, 2, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 1,
       0, 2, 0, 2, 0, 2, 2, 1, 2, 0, 0, 2, 1, 0, 2, 2, 1, 0, 2, 2, 2, 1,
       1, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 1, 1, 1, 2, 2, 0, 0,
       0, 2, 0, 1, 2, 2, 2, 0, 2, 1, 1, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2,
       2, 1, 0, 2, 1, 0, 1, 2, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 2,
       0, 1, 2, 0, 0, 1, 2, 1, 0, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 0, 2, 2,
       0, 2, 0, 0])

In [100]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,NRS260,0,1
1,NRS148,2,2
2,NRS205,1,2
3,NRS064,1,2
4,NRS209,2,2
...,...,...,...
197,NRS255,2,2
198,GA53649,0,0
199,NRS209,2,2
200,NRS210,0,0


In [101]:
proba7 = model1_over7.predict_proba(X_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [102]:
dat_proba7

,0,1,2
0,1.910703e-01,8.089237e-01,5.899579e-06
1,6.019913e-09,9.196866e-07,9.999990e-01
2,3.036906e-05,1.417140e-03,9.985525e-01
3,5.137212e-03,1.493428e-01,8.455200e-01
4,4.863225e-10,1.086092e-07,9.999999e-01
...,...,...,...
197,5.137212e-03,1.493428e-01,8.455200e-01
198,9.999998e-01,1.917346e-07,5.435073e-11
199,4.863225e-10,1.086092e-07,9.999999e-01
200,9.999999e-01,6.615954e-08,3.265495e-21


In [103]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [104]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p003ST.csv", index = False,
         header=None)

In [113]:
hist1_over7 = model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 191us/step - loss: 1.2601 - accuracy: 0.7447 - val_loss: 1.0064 - val_accuracy: 0.9208
Epoch 2/100
470/470 [==============================] - 0s 151us/step - loss: 1.2713 - accuracy: 0.7596 - val_loss: 1.0617 - val_accuracy: 0.9257
Epoch 3/100
470/470 [==============================] - 0s 127us/step - loss: 1.4613 - accuracy: 0.7255 - val_loss: 0.9513 - val_accuracy: 0.9307
Epoch 4/100
470/470 [==============================] - 0s 129us/step - loss: 1.2524 - accuracy: 0.7447 - val_loss: 0.9522 - val_accuracy: 0.9158
Epoch 5/100
470/470 [==============================] - 0s 187us/step - loss: 1.2148 - accuracy: 0.7511 - val_loss: 0.9698 - val_accuracy: 0.9158
Epoch 6/100
470/470 [==============================] - 0s 198us/step - loss: 1.2385 - accuracy: 0.7532 - val_loss: 0.8065 - val_accuracy: 0.9257
Epoch 7/100
470/470 [==============================] - 0s 188us/step - loss: 1.2186 

In [114]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over7.history['accuracy'])*100))

over-sampling train accuracy: 76.14%


In [166]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [167]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
1,p002ykpresabsSTCC_qual,BCH-SA-09,1,1,1.137139e-06,9.999988e-01,2.067601e-09
2,p002ykpresabsSTCC_qual,NRS224,0,0,1.000000e+00,2.093110e-31,0.000000e+00
3,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
4,p002ykpresabsSTCC_qual,NRS235,1,1,2.243513e-02,9.774035e-01,1.615106e-04
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS035,0,0,9.354528e-01,6.414209e-02,4.051121e-04
1978,pyopresabsSTCC_qual,NRS260,1,1,4.808470e-08,1.000000e+00,7.364639e-09
1979,pyopresabsSTCC_qual,CA9,0,0,1.000000e+00,2.361323e-08,2.871247e-08
1980,pyopresabsSTCC_qual,NRS183,1,1,2.755864e-07,9.999998e-01,5.310879e-08


In [168]:
y_prob7 = df_proba7[df_proba7['phage']=='p003pkpresabsSTCC_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[1.91070330e-01, 8.08923700e-01, 5.89957900e-06],
       [6.01991260e-09, 9.19686600e-07, 9.99999050e-01],
       [3.03690610e-05, 1.41713950e-03, 9.98552500e-01],
       [5.13721200e-03, 1.49342820e-01, 8.45519960e-01],
       [4.86322500e-10, 1.08609250e-07, 9.99999900e-01],
       [2.02612520e-05, 9.99979600e-01, 8.02177240e-08],
       [9.95574000e-01, 4.42594600e-03, 7.46347800e-12],
       [2.02612520e-05, 9.99979600e-01, 8.02177240e-08],
       [4.86322500e-10, 1.08609250e-07, 9.99999900e-01],
       [1.00000000e+00, 1.20013240e-08, 1.31695870e-09],
       [9.99999900e-01, 6.61595440e-08, 3.26549480e-21],
       [1.48641620e-05, 9.99979600e-01, 5.44671900e-06],
       [4.86322500e-10, 1.08609250e-07, 9.99999900e-01],
       [1.33532580e-06, 9.99998570e-01, 1.49393510e-07],
       [5.13721200e-03, 1.49342820e-01, 8.45519960e-01],
       [9.95574000e-01, 4.42594600e-03, 7.46347800e-12],
       [6.01991260e-09, 9.19686600e-07, 9.99999050e-01],
       [9.99998800e-01, 1.15188

In [169]:
ovo7 = rocauc_ovo(y_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.9725885821437475

In [170]:
ovr7 = rocauc_ovr(y_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.9725885821437475

In [171]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_over)

In [172]:
dat8 = pd.DataFrame(X_test_over[:,0])
dat8['test'] = y_test_over

In [173]:
dat8

,0,test
0,CA541,1
1,SR3585,0
2,NRS232,1
3,NRS148,2
4,NRS180,1
...,...,...
197,NRS209,2
198,NRS035,1
199,506,0
200,SR2091,0


In [174]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [109]:
model1_over8 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [110]:
model1_over8.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 890us/step - loss: 9.1235 - accuracy: 0.3851 - val_loss: 5.4841 - val_accuracy: 0.6188
Epoch 2/100
470/470 [==============================] - 0s 200us/step - loss: 6.3987 - accuracy: 0.5255 - val_loss: 3.6354 - val_accuracy: 0.5743
Epoch 3/100
470/470 [==============================] - 0s 255us/step - loss: 5.0567 - accuracy: 0.5553 - val_loss: 2.4884 - val_accuracy: 0.6139
Epoch 4/100
470/470 [==============================] - 0s 189us/step - loss: 4.2149 - accuracy: 0.5766 - val_loss: 1.9187 - val_accuracy: 0.6337
Epoch 5/100
470/470 [==============================] - 0s 167us/step - loss: 3.8136 - accuracy: 0.5766 - val_loss: 1.5601 - val_accuracy: 0.6040
Epoch 6/100
470/470 [==============================] - 0s 270us/step - loss: 3.9670 - accuracy: 0.5851 - val_loss: 1.2235 - val_accuracy: 0.6485
Epoch 7/100
470/470 [==============================] - 0s 237us/step - loss: 3.1748 

Epoch 57/100
470/470 [==============================] - 0s 183us/step - loss: 2.0712 - accuracy: 0.7383 - val_loss: 1.2173 - val_accuracy: 0.8614
Epoch 58/100
470/470 [==============================] - 0s 137us/step - loss: 1.8811 - accuracy: 0.7383 - val_loss: 1.5324 - val_accuracy: 0.8515
Epoch 59/100
470/470 [==============================] - 0s 122us/step - loss: 1.9498 - accuracy: 0.7255 - val_loss: 1.3068 - val_accuracy: 0.8960
Epoch 60/100
470/470 [==============================] - 0s 214us/step - loss: 1.8532 - accuracy: 0.7532 - val_loss: 1.3347 - val_accuracy: 0.8812
Epoch 61/100
470/470 [==============================] - 0s 175us/step - loss: 1.8258 - accuracy: 0.7234 - val_loss: 1.0622 - val_accuracy: 0.8812
Epoch 62/100
470/470 [==============================] - 0s 129us/step - loss: 1.8467 - accuracy: 0.7553 - val_loss: 0.9187 - val_accuracy: 0.8861
Epoch 63/100
470/470 [==============================] - 0s 121us/step - loss: 1.9101 - accuracy: 0.7298 - val_loss: 1.0193 -

In [122]:
acc_test_over8 = model1_over8.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over8*100))

202/202 [==============================] - 0s 111us/step
over-sampling test accuracy: 92.57%


In [112]:
pred8 = model1_over8.predict_classes(X_test_over)
pred8

array([0, 0, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 0, 2, 1, 1, 1, 0, 0, 2,
       1, 0, 2, 0, 1, 1, 1, 1, 2, 1, 0, 0, 1, 0, 1, 2, 2, 0, 2, 2, 2, 2,
       2, 2, 0, 0, 2, 2, 0, 1, 1, 0, 0, 2, 0, 2, 1, 1, 2, 0, 2, 1, 0, 2,
       0, 1, 0, 2, 2, 0, 2, 1, 0, 2, 2, 2, 1, 2, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 2, 2, 0, 1, 1, 0, 0, 0, 2, 0, 1, 2, 0, 2, 2, 1,
       0, 2, 2, 2, 2, 1, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 1, 2, 2, 1, 2, 0,
       1, 2, 0, 0, 2, 2, 2, 2, 0, 1, 2, 2, 2, 0, 1, 0, 1, 2, 1, 0, 0, 2,
       1, 0, 0, 0, 2, 1, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 0, 0, 2,
       2, 1, 2, 0, 1, 2, 2, 1, 0, 2, 1, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2,
       1, 0, 1, 2])

In [113]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,CA541,1,0
1,SR3585,0,0
2,NRS232,1,1
3,NRS148,2,2
4,NRS180,1,1
...,...,...,...
197,NRS209,2,2
198,NRS035,1,1
199,506,0,0
200,SR2091,0,1


In [114]:
proba8 = model1_over8.predict_proba(X_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [115]:
dat_proba8

,0,1,2
0,9.969602e-01,3.039824e-03,8.615204e-10
1,9.969602e-01,3.039824e-03,8.615204e-10
2,1.787695e-06,9.999982e-01,3.214948e-10
3,4.319543e-08,1.008986e-07,9.999999e-01
4,4.011245e-10,1.000000e+00,4.781401e-11
...,...,...,...
197,1.449390e-09,6.278701e-09,1.000000e+00
198,4.906269e-10,1.000000e+00,4.059978e-13
199,9.999999e-01,9.207479e-08,9.831998e-12
200,4.492901e-05,9.999549e-01,1.165648e-07


In [116]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [117]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p003ST.csv", index = False,
         header=None)

In [126]:
hist1_over8 = model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 211us/step - loss: 1.4766 - accuracy: 0.7106 - val_loss: 1.0298 - val_accuracy: 0.8812
Epoch 2/100
470/470 [==============================] - 0s 182us/step - loss: 1.2216 - accuracy: 0.7426 - val_loss: 0.7255 - val_accuracy: 0.9109
Epoch 3/100
470/470 [==============================] - 0s 175us/step - loss: 1.1136 - accuracy: 0.7723 - val_loss: 0.6541 - val_accuracy: 0.9059
Epoch 4/100
470/470 [==============================] - 0s 188us/step - loss: 1.1265 - accuracy: 0.7660 - val_loss: 0.6172 - val_accuracy: 0.9109
Epoch 5/100
470/470 [==============================] - 0s 186us/step - loss: 1.0156 - accuracy: 0.7617 - val_loss: 0.8317 - val_accuracy: 0.9208
Epoch 6/100
470/470 [==============================] - 0s 183us/step - loss: 1.1365 - accuracy: 0.7553 - val_loss: 0.5337 - val_accuracy: 0.9109
Epoch 7/100
470/470 [==============================] - 0s 206us/step - loss: 1.2868 

Epoch 57/100
470/470 [==============================] - 0s 122us/step - loss: 0.9610 - accuracy: 0.7553 - val_loss: 0.4524 - val_accuracy: 0.9208
Epoch 58/100
470/470 [==============================] - 0s 118us/step - loss: 0.9293 - accuracy: 0.7532 - val_loss: 0.6538 - val_accuracy: 0.9455
Epoch 59/100
470/470 [==============================] - 0s 135us/step - loss: 1.0819 - accuracy: 0.7404 - val_loss: 0.7406 - val_accuracy: 0.9455
Epoch 60/100
470/470 [==============================] - 0s 155us/step - loss: 1.1219 - accuracy: 0.7447 - val_loss: 0.6481 - val_accuracy: 0.9455
Epoch 61/100
470/470 [==============================] - 0s 401us/step - loss: 0.7475 - accuracy: 0.8149 - val_loss: 0.4204 - val_accuracy: 0.9307
Epoch 62/100
470/470 [==============================] - 0s 191us/step - loss: 1.2104 - accuracy: 0.7660 - val_loss: 0.7108 - val_accuracy: 0.9406
Epoch 63/100
470/470 [==============================] - 0s 191us/step - loss: 0.9307 - accuracy: 0.7957 - val_loss: 0.6432 -

In [127]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over8.history['accuracy'])*100))

over-sampling train accuracy: 75.96%


In [175]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [176]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,CFBREBSa116,0,0,9.676203e-01,3.237956e-02,1.480166e-07
1,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,6.534852e-11,2.250731e-18
2,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
3,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
4,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS205,2,2,3.691095e-08,3.571927e-08,9.999999e-01
1978,pyopresabsSTCC_qual,CFBREBSa122,0,1,9.261665e-02,9.073822e-01,1.162373e-06
1979,pyopresabsSTCC_qual,NRS001,1,1,4.174278e-07,9.999995e-01,3.254024e-09
1980,pyopresabsSTCC_qual,NRS148,2,2,3.234670e-08,3.121212e-08,9.999999e-01


In [177]:
y_prob8 = df_proba8[df_proba8['phage']=='p003pkpresabsSTCC_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[9.96960160e-01, 3.03982430e-03, 8.61520400e-10],
       [9.96960160e-01, 3.03982430e-03, 8.61520400e-10],
       [1.78769460e-06, 9.99998200e-01, 3.21494800e-10],
       [4.31954350e-08, 1.00898570e-07, 9.99999900e-01],
       [4.01124500e-10, 1.00000000e+00, 4.78140130e-11],
       [4.31954350e-08, 1.00898570e-07, 9.99999900e-01],
       [1.81552200e-03, 2.62257100e-03, 9.95561960e-01],
       [2.20574830e-04, 4.21209580e-04, 9.99358240e-01],
       [2.52445300e-17, 1.00000000e+00, 3.75606500e-15],
       [4.38614620e-05, 9.99956000e-01, 1.11552330e-07],
       [1.05748220e-08, 1.00000000e+00, 2.73962910e-14],
       [7.28439900e-05, 9.99927160e-01, 1.29792440e-08],
       [1.61516420e-08, 1.00000000e+00, 3.93807460e-10],
       [1.44938970e-09, 6.27870100e-09, 1.00000000e+00],
       [1.00000000e+00, 3.70838720e-10, 1.16078610e-15],
       [4.31954350e-08, 1.00898570e-07, 9.99999900e-01],
       [7.41889300e-12, 1.00000000e+00, 4.46696200e-10],
       [7.28439900e-05, 9.99927

In [178]:
ovo8 = rocauc_ovo(y_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.9539199427698111

In [179]:
ovr8 = rocauc_ovr(y_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.9539199427698111

In [180]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.9676615221821178

In [181]:
np.std(ovos2)

0.008664610735943193

In [182]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.9676615221821178

In [183]:
np.std(ovrs2)

0.008664610735943193

In [128]:
accs_reg = [acc_test_over5, acc_test_over6, acc_test_over7, acc_test_over8]

In [129]:
mean_reg = np.mean(accs_reg)
print('over-sampling test accuracy regularization mean: %.2f%%' % (mean_reg*100))

over-sampling test accuracy regularization mean: 90.97%


In [130]:
std_reg = np.std(accs_reg)
print('over-sampling test accuracy regularization standard deviation:', std_reg)

over-sampling test accuracy regularization standard deviation: 0.017282487758180157


In [131]:
accs_train_reg = [np.mean(hist1_over5.history['accuracy']), np.mean(hist1_over6.history['accuracy']), np.mean(hist1_over7.history['accuracy']),
             np.mean(hist1_over8.history['accuracy'])]

In [132]:
mean_train_reg = np.mean(accs_train_reg)
print('over-sampling train accuracy regularization mean: %.2f%%' % (mean_train_reg*100))

over-sampling train accuracy regularization mean: 75.79%


In [133]:
std_train_reg = np.std(accs_train_reg)
print('over-sampling train accuracy regularization standard deviation:', std_train_reg)

over-sampling train accuracy regularization standard deviation: 0.0030917057
